In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1648112337295_0005,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
MoviesParquet_node1648104136489 = glueContext.create_dynamic_frame.from_catalog(
    database="gks_db",
    table_name="silver_movies",
    transformation_ctx="MoviesParquet_node1648104136489",
)

MoviesParquet_node1648104136489.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- movieid: long
|-- title: string
|-- genres: string

In [3]:
movieDf = MoviesParquet_node1648104136489.toDF()
movieDf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------------------+
|movieid|               title|              genres|
+-------+--------------------+--------------------+
| 103609|    Class Act (1992)|              Comedy|
|    990| Maximum Risk (1996)|Action|Adventure|...|
|   2362|Glen or Glenda (1...|               Drama|
|   4642|Hedwig and the An...|Comedy|Drama|Musical|
|  47382|      Step Up (2006)|       Drama|Romance|
|  26285|    Dark Star (1974)|Comedy|Sci-Fi|Thr...|
|  72701|    Planet 51 (2009)|Adventure|Animati...|
|   5334| Cadillac Man (1990)|        Comedy|Crime|
|   2388|Steam: The Turkis...|       Drama|Romance|
| 135198|The Hairdresser (...|        Comedy|Drama|
|  51412|         Next (2007)|Action|Sci-Fi|Thr...|
|   4321|City Slickers (1991)|      Comedy|Western|
|   3843|Sleepaway Camp (1...|              Horror|
|   8873|Motorcycle Diarie...|     Adventure|Drama|
| 103075|   Purge, The (2013)|Crime|Horror|Thri...|
|  39444|Weather Man, The ...|        Comedy|Drama|
|  26464| Bl

In [4]:
RatingsParquet_node1648104140409 = glueContext.create_dynamic_frame.from_catalog(
    database="gks_db",
    table_name="silver_ratings",
    transformation_ctx="RatingsParquet_node1648104140409",
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
RatingsParquet_node1648104140409.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- userid: long
|-- movieid: long
|-- rating: double
|-- timestamp: long

In [6]:

# Script generated for node movies
movies_node1648104467325 = ApplyMapping.apply(
    frame=MoviesParquet_node1648104136489,
    mappings=[
        ("movieid", "long", "movie_id", "long"),
        ("title", "string", "name", "string"),
    ],
    transformation_ctx="movies_node1648104467325",
)
movies_node1648104467325.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- movie_id: long
|-- name: string

In [7]:
movies_node1648104467325.toDF().show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+
|movie_id|                name|
+--------+--------------------+
|  103609|    Class Act (1992)|
|     990| Maximum Risk (1996)|
|    2362|Glen or Glenda (1...|
+--------+--------------------+
only showing top 3 rows

In [8]:
ApplyMapping_node1648104492447 = ApplyMapping.apply(
    frame=RatingsParquet_node1648104140409,
    mappings=[
        ("userid", "long", "userid", "long"),
        ("movieid", "long", "mid", "long"),
        ("rating", "double", "rating", "double"),
    ],
    transformation_ctx="ApplyMapping_node1648104492447",
)

ApplyMapping_node1648104492447.printSchema()
ApplyMapping_node1648104492447.toDF().show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- userid: long
|-- mid: long
|-- rating: double

+------+-----+------+
|userid|  mid|rating|
+------+-----+------+
|   610|84772|   3.5|
|   468|  317|   3.0|
+------+-----+------+
only showing top 2 rows

In [9]:
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql import functions as SqlFuncs

def sparkAggregate(
    glueContext, parentFrame, groups, aggs, transformation_ctx
) -> DynamicFrame:
    aggsFuncs = []
    for column, func in aggs:
        aggsFuncs.append(getattr(SqlFuncs, func)(column))
    result = (
        parentFrame.toDF().groupBy(*groups).agg(*aggsFuncs)
        if len(groups) > 0
        else parentFrame.toDF().agg(*aggsFuncs)
    )
    return DynamicFrame.fromDF(result, glueContext, transformation_ctx)


# Script generated for node Aggregate
Aggregate_node1648104595189 = sparkAggregate(
    glueContext,
    parentFrame=ApplyMapping_node1648104492447,
    groups=["mid"],
    aggs=[["userid", "count"], ["rating", "avg"],["rating", "min"], ["rating", "max"]  ],
    transformation_ctx="Aggregate_node1648104595189",
)


Aggregate_node1648104595189.printSchema()
Aggregate_node1648104595189.toDF().show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- mid: long
|-- count(userid): long
|-- avg(rating): double
|-- min(rating): double
|-- max(rating): double

+------+-------------+------------------+-----------+-----------+
|   mid|count(userid)|       avg(rating)|min(rating)|max(rating)|
+------+-------------+------------------+-----------+-----------+
|102852|            1|               3.0|        3.0|        3.0|
|106100|           17|3.9705882352941178|        0.5|        5.0|
+------+-------------+------------------+-----------+-----------+
only showing top 2 rows

In [10]:
apply_renamed_agg_node = ApplyMapping.apply(
    frame=Aggregate_node1648104595189,
    mappings=[
         ("mid", "long", "mid", "long"),
        ("count(userid)", "long", "total_ratings", "long"),
        ("avg(rating)", "double", "avg_rating", "double"),
         ("min(rating)", "double", "min_rating", "double"),
         ("max(rating)", "double", "max_rating", "double"),
    ],
    transformation_ctx="apply_renamed_agg_node",
)

apply_renamed_agg_node.printSchema()
apply_renamed_agg_node.toDF().show(2)
 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- mid: long
|-- total_ratings: long
|-- avg_rating: double
|-- min_rating: double
|-- max_rating: double

+------+-------------+------------------+----------+----------+
|   mid|total_ratings|        avg_rating|min_rating|max_rating|
+------+-------------+------------------+----------+----------+
|102852|            1|               3.0|       3.0|       3.0|
|106100|           17|3.9705882352941178|       0.5|       5.0|
+------+-------------+------------------+----------+----------+
only showing top 2 rows

In [11]:
Join_node1648105003564 = Join.apply(
    frame1=apply_renamed_agg_node,
    frame2=movies_node1648104467325,
    keys1=["mid"],
    keys2=["movie_id"],
    transformation_ctx="Join_node1648105003564",
)


Join_node1648105003564.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- mid: long
|-- total_ratings: long
|-- movie_id: long
|-- min_rating: double
|-- name: string
|-- max_rating: double
|-- avg_rating: double

In [14]:
Join_node1648105003564.toDF().show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------------+--------+----------+--------------------+----------+----------+
| mid|total_ratings|movie_id|min_rating|                name|max_rating|avg_rating|
+----+-------------+--------+----------+--------------------+----------+----------+
|6688|            1|    6688|       3.5|Autumn Spring (Ba...|       3.5|       3.5|
|6237|            1|    6237|       4.0|Glenn Miller Stor...|       4.0|       4.0|
+----+-------------+--------+----------+--------------------+----------+----------+
only showing top 2 rows

In [15]:

# Script generated for node Drop Fields
DropFields_node1648105395080 = DropFields.apply(
    frame=Join_node1648105003564,
    paths=["mid"],
    transformation_ctx="DropFields_node1648105395080",
)

# Script generated for node Filter
Filter_node1648105461556 = Filter.apply(
    frame=DropFields_node1648105395080,
    f=lambda row: (row["total_ratings"] >= 100 and row["avg_rating"] >= 3.5),
    transformation_ctx="Filter_node1648105461556",
)

Filter_node1648105461556.printSchema()
Filter_node1648105461556.toDF().show(4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- total_ratings: long
|-- movie_id: long
|-- min_rating: double
|-- name: string
|-- max_rating: double
|-- avg_rating: double

+-------------+--------+----------+--------------------+----------+-----------------+
|total_ratings|movie_id|min_rating|                name|max_rating|       avg_rating|
+-------------+--------+----------+--------------------+----------+-----------------+
|          122|    5349|       0.5|   Spider-Man (2002)|       5.0|3.540983606557377|
|          129|    1221|       1.0|Godfather: Part I...|       5.0| 4.25968992248062|
|          102|    1222|       1.0|Full Metal Jacket...|       5.0|4.098039215686274|
|          108|    2115|       1.0|Indiana Jones and...|       5.0|3.638888888888889|
+-------------+--------+----------+--------------------+----------+-----------------+
only showing top 4 rows

In [16]:
Filter_node1648105461556 = Filter_node1648105461556.coalesce(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:

# Script generated for node S3 bucket
S3bucket_node3 = glueContext.write_dynamic_frame.from_options(
    frame=Filter_node1648105461556,
    connection_type="s3",
    format="glueparquet",
    connection_options={"path": "s3://gks-bucket/gold/popular-movies-copy3/", "partitionKeys": []},
    format_options={"compression": "snappy"},
    transformation_ctx="Filter_node1648105461556",
)
 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:

# job.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'job' is not defined
Traceback (most recent call last):
NameError: name 'job' is not defined

